In [ ]:
def split_audio(input_folder, output_folder, segment_length=2000, min_amplitude_db=-30):
    input_folder_path = Path(input_folder)
    output_folder_path = Path(output_folder)

    file_counter = 0
    for file in input_folder_path.rglob('*'):
        if file.is_file() and file.suffix.lower() in {'.wav', '.mp3'}:
            file_counter += 1
            try:
                relative_path = file.relative_to(input_folder_path)

                output_subfolder = output_folder_path / relative_path.parent
                output_subfolder.mkdir(parents=True, exist_ok=True)

                audio = AudioSegment.from_file(str(file), format=file.suffix.lower()[1:])
                audio = reduce_noise_deep_filter(audio, noise_reduction_factor=0.5)

                for i, chunk in enumerate(audio[::segment_length]):
                    if chunk.dBFS > min_amplitude_db:
                        output_file = output_subfolder / f"{relative_path.parent}_{file_counter}_{i}.wav"
                        chunk.export(output_file, format=file.suffix.lower()[1:])
            except Exception as e:
                print(f"Error processing {file}: {str(e)}")